# 04 — Score YouTube Comments + MC Dropout Uncertainty

Loads the trained RoBERTa model and runs it over `data/processed/comments_clean.parquet`.

**Outputs per comment:**
- `score_toxicity`, `score_hate_racism`, `score_harassment` — mean probability across MC samples
- `flag_toxicity`, `flag_hate_racism`, `flag_harassment` — binary decision at threshold
- `uncertainty_epistemic` — variance across MC Dropout samples (how unsure the model is)
- `mc_samples` — raw T=10 score array (JSON)

**Silver labels** = high-confidence predictions (score > 0.8 or score < 0.2).  
**High uncertainty** = candidates for human labeling (notebook 05 / M4).

In [ ]:
# ── CONFIG ─────────────────────────────────────────────────────────
# QUICK_TEST=True  → 500 comments, 3 MC passes, fast CPU check (~30s)
# QUICK_TEST=False → ALL comments, 10 MC passes (run on GPU/Colab)
# SCORE_ALL=True   → ALL comments, 3 MC passes, CPU overnight (~2-3h)
QUICK_TEST   = True
SCORE_ALL    = True   # Set True + QUICK_TEST=False to score all 130k on CPU

THRESHOLD    = 0.5
SILVER_HIGH  = 0.8
SILVER_LOW   = 0.2

# Settings per mode
if QUICK_TEST:
    MC_SAMPLES   = 3
    BATCH_SIZE   = 64
    MAX_LENGTH   = 64
    MAX_COMMENTS = 500
elif SCORE_ALL:
    MC_SAMPLES   = 3    # Fewer passes = faster (still gives good uncertainty estimate)
    BATCH_SIZE   = 64
    MAX_LENGTH   = 64
    MAX_COMMENTS = None  # Score everything
else:  # Full GPU mode
    MC_SAMPLES   = 10
    BATCH_SIZE   = 32
    MAX_LENGTH   = 128
    MAX_COMMENTS = None  # Score everything
# ───────────────────────────────────────────────────────────────────
print(f"Mode: {'QUICK_TEST' if QUICK_TEST else 'SCORE_ALL' if SCORE_ALL else 'FULL GPU'}")
print(f"MC_SAMPLES={MC_SAMPLES}, MAX_LENGTH={MAX_LENGTH}, MAX_COMMENTS={MAX_COMMENTS}")

Mode: QUICK_TEST
MC_SAMPLES=3, MAX_LENGTH=64, MAX_COMMENTS=500


In [28]:
import sys, json, warnings
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import plotly.express as px
from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaTokenizerFast, RobertaModel

warnings.filterwarnings("ignore")
sys.path.insert(0, str(Path("..").resolve()))

ROOT      = Path("..").resolve()
MODEL_DIR = ROOT / "models"
DATA_DIR  = ROOT / "data" / "processed"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {DEVICE}")

LABEL_COLS = ["label_toxicity", "label_hate_racism", "label_harassment"]
SCORE_COLS = [c.replace("label_", "score_") for c in LABEL_COLS]
FLAG_COLS  = [c.replace("label_", "flag_")  for c in LABEL_COLS]

Device: cpu


## 1. Load Model

In [29]:
class ToxicityClassifier(nn.Module):
    def __init__(self, model_name: str, num_labels: int = 3, dropout: float = 0.3):
        super().__init__()
        self.roberta    = RobertaModel.from_pretrained(model_name)
        hidden          = self.roberta.config.hidden_size
        self.dropout    = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden, num_labels)

    def forward(self, input_ids, attention_mask):
        out = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        cls = out.last_hidden_state[:, 0, :]
        return self.classifier(self.dropout(cls))

# Load metadata
with open(MODEL_DIR / "model_meta.json") as f:
    meta = json.load(f)
print("Model meta:", meta)

model = ToxicityClassifier(meta["model_name"]).to(DEVICE)
model.load_state_dict(torch.load(
    MODEL_DIR / "roberta_toxicity_best.pt",
    map_location=DEVICE,
    weights_only=True,
))
tokenizer = RobertaTokenizerFast.from_pretrained(MODEL_DIR / "tokenizer")
print("Model loaded.")

Model meta: {'model_name': 'roberta-base', 'label_cols': ['label_toxicity', 'label_hate_racism', 'label_harassment'], 'max_length': 128, 'best_avg_f1': 0.5781, 'quick_test': True, 'brier_toxicity': 0.04568375647068024, 'brier_hate_racism': 0.22646364569664001, 'brier_harassment': 0.19959355890750885, 'ece_toxicity': 0.0587, 'ece_hate_racism': 0.2624, 'ece_harassment': 0.2187}


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaModel LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
lm_head.bias                    | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
pooler.dense.weight             | MISSING    | 
pooler.dense.bias               | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Model loaded.


## 2. Load YouTube Comments

In [30]:
df = pd.read_parquet(DATA_DIR / "comments_clean.parquet")

if MAX_COMMENTS:
    df = df.head(MAX_COMMENTS).copy()
    print(f"QUICK_TEST: scoring first {MAX_COMMENTS} comments")

print(f"Loaded {len(df):,} comments from comments_clean.parquet")

QUICK_TEST: scoring first 500 comments
Loaded 500 comments from comments_clean.parquet


## 3. Inference with MC Dropout

In [31]:
class TextDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts   = texts
        self.tok     = tokenizer
        self.max_len = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = self.tok(
            self.texts[idx], max_length=self.max_len,
            padding="max_length", truncation=True, return_tensors="pt",
        )
        return {
            "input_ids":      enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
        }


def mc_dropout_predict(model, loader, device, T=10):
    """
    Run T stochastic forward passes with dropout active.
    Returns:
        means     — shape (N, 3)
        variances — shape (N, 3)  epistemic uncertainty
        all_probs — shape (T, N, 3) raw samples
    """
    model.train()   # keep dropout ON during inference
    all_runs = []

    with torch.no_grad():
        for t in range(T):
            run_probs = []
            for batch in loader:
                ids  = batch["input_ids"].to(device)
                mask = batch["attention_mask"].to(device)
                probs = torch.sigmoid(model(ids, mask)).cpu().numpy()
                run_probs.append(probs)
            all_runs.append(np.vstack(run_probs))
            print(f"  MC pass {t+1}/{T} done")

    all_probs = np.stack(all_runs)
    return all_probs.mean(axis=0), all_probs.var(axis=0), all_probs


dataset = TextDataset(df["text_clean"].tolist(), tokenizer, MAX_LENGTH)
loader  = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"Running {MC_SAMPLES} MC Dropout passes over {len(df):,} comments "
      f"(batch={BATCH_SIZE}, max_len={MAX_LENGTH})...")

means, variances, all_probs = mc_dropout_predict(model, loader, DEVICE, T=MC_SAMPLES)
print("Done.")

Running 3 MC Dropout passes over 500 comments (batch=64, max_len=64)...
  MC pass 1/3 done
  MC pass 2/3 done
  MC pass 3/3 done
Done.


## 4. Attach Scores to DataFrame

In [32]:
scored = df.copy()

# Mean scores per label
for i, col in enumerate(SCORE_COLS):
    scored[col] = means[:, i].round(4)

# Binary flags
for i, col in enumerate(FLAG_COLS):
    scored[col] = (means[:, i] >= THRESHOLD).astype(int)

# Epistemic uncertainty = variance across MC passes (how unsure the model is)
# Shape: (T, N, 3) → var over T → mean over labels → (N,)
scored["uncertainty_epistemic"] = variances.mean(axis=1).round(6)

# Aleatoric uncertainty = mean(p*(1-p)) across MC passes (inherent label noise)
# Captures ambiguity in the data itself, not just model uncertainty
# Shape: (T, N, 3) → p*(1-p) → mean over T → mean over labels → (N,)
aleatoric_per_label = (all_probs * (1 - all_probs)).mean(axis=0)   # (N, 3)
scored["uncertainty_aleatoric"] = aleatoric_per_label.mean(axis=1).round(6)

# MC samples as JSON list (for BigQuery storage later)
scored["mc_samples"] = [
    json.dumps(all_probs[:, i, :].tolist())
    for i in range(len(df))
]

scored["model_version"] = f"{meta['model_name']}_quick={meta['quick_test']}"

print(f"Scored {len(scored):,} comments")
print("\nFlag rates:")
for col in FLAG_COLS:
    n = scored[col].sum()
    print(f"  {col}: {n:,} flagged ({n/len(scored)*100:.1f}%)")

print(f"\nEpistemic uncertainty — mean: {scored['uncertainty_epistemic'].mean():.5f}  "
      f"max: {scored['uncertainty_epistemic'].max():.5f}")
print(f"Aleatoric uncertainty — mean: {scored['uncertainty_aleatoric'].mean():.5f}  "
      f"max: {scored['uncertainty_aleatoric'].max():.5f}")

Scored 500 comments

Flag rates:
  flag_toxicity: 26 flagged (5.2%)
  flag_hate_racism: 15 flagged (3.0%)
  flag_harassment: 20 flagged (4.0%)

Epistemic uncertainty — mean: 0.00207  max: 0.09491
Aleatoric uncertainty — mean: 0.02278  max: 0.22172


## 4b. Sentiment Analysis (VADER)

Positive sentiment comments cannot be toxic — use this as a fast, GPU-free pre-filter to eliminate false positives.

In [ ]:
import nltk
nltk.download('vader_lexicon', quiet=True)
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()

def classify_sentiment(text: str) -> tuple:
    """Return (label, compound_score). Thresholds follow standard VADER convention."""
    scores = sia.polarity_scores(str(text))
    c = scores["compound"]
    if c >= 0.05:
        return "positive", round(c, 4)
    elif c <= -0.05:
        return "negative", round(c, 4)
    else:
        return "neutral", round(c, 4)

print("Computing VADER sentiment for all comments...")
sentiment_results        = scored["text_clean"].apply(classify_sentiment)
scored["sentiment"]      = sentiment_results.apply(lambda x: x[0])
scored["sentiment_score"] = sentiment_results.apply(lambda x: x[1])

print("\nSentiment distribution:")
print(scored["sentiment"].value_counts())
print(f"\nMean compound score: {scored['sentiment_score'].mean():.3f}")

# ── Sentiment-based flag filter ──────────────────────────────────────────────
# A positive-sentiment comment cannot be toxic. Zero out flags for positive comments.
# This eliminates false positives (e.g. "So sorry 🙏", "Love her response", "Long live Persia 🎉")
APPLY_SENTIMENT_FILTER = True

if APPLY_SENTIMENT_FILTER:
    is_positive = scored["sentiment"] == "positive"
    before_flags = scored[FLAG_COLS].sum()
    scored.loc[is_positive, FLAG_COLS] = 0
    after_flags = scored[FLAG_COLS].sum()

    print("\nSentiment filter applied (positive → flags set to 0):")
    for col in FLAG_COLS:
        removed = int(before_flags[col] - after_flags[col])
        name = col.replace("flag_", "")
        print(f"  {name}: {int(before_flags[col])} → {int(after_flags[col])} flagged  ({removed} removed)")
    print(f"\nPositive comments filtered: {is_positive.sum():,} / {len(scored):,} ({is_positive.mean()*100:.1f}%)")

## 5. Silver Labels

In [33]:
# Silver = high-confidence predictions on any label
is_silver_pos = (scored[SCORE_COLS] >= SILVER_HIGH).any(axis=1)
is_silver_neg = (scored[SCORE_COLS] <= SILVER_LOW).all(axis=1)
scored["is_silver"] = (is_silver_pos | is_silver_neg).astype(int)

silver = scored[scored["is_silver"] == 1]
print(f"Silver labels: {len(silver):,} / {len(scored):,} ({len(silver)/len(scored)*100:.1f}%)")
print(f"  High-confidence positive: {is_silver_pos.sum():,}")
print(f"  High-confidence negative: {is_silver_neg.sum():,}")

Silver labels: 467 / 500 (93.4%)
  High-confidence positive: 13
  High-confidence negative: 454


## 6. Visualise Results

In [34]:
# Score distributions
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=1, cols=3, subplot_titles=SCORE_COLS)
colors = ["steelblue", "darkorange", "mediumseagreen"]
for i, (col, color) in enumerate(zip(SCORE_COLS, colors), 1):
    fig.add_trace(go.Histogram(x=scored[col], nbinsx=50, name=col,
                               marker_color=color), row=1, col=i)
fig.update_layout(title="Score Distributions (sigmoid probabilities)", showlegend=False)
fig.show()

In [35]:
# Toxicity by channel
channel_scores = scored.groupby("channel_name")["score_toxicity"].mean().sort_values(ascending=False)
fig2 = px.bar(channel_scores.reset_index(), x="channel_name", y="score_toxicity",
              title="Mean Toxicity Score by Channel",
              labels={"channel_name": "Channel", "score_toxicity": "Avg Toxicity Score"},
              color="score_toxicity", color_continuous_scale="Reds")
fig2.update_layout(xaxis_tickangle=-20)
fig2.show()

In [36]:
# Highest uncertainty comments — candidates for human labeling (M4)
uncertain = scored.nlargest(10, "uncertainty_epistemic")[
    ["channel_name", "text_clean", "score_toxicity", "score_hate_racism",
     "score_harassment", "uncertainty_epistemic", "uncertainty_aleatoric"]
]
pd.set_option("display.max_colwidth", 80)
pd.set_option("display.float_format", "{:.4f}".format)
print("Top 10 highest-uncertainty comments (prime candidates for human labeling):")
uncertain

Top 10 highest-uncertainty comments (prime candidates for human labeling):


,channel_name,text_clean,score_toxicity,score_hate_racism,score_harassment,uncertainty_epistemic,uncertainty_aleatoric
434,Fox News,"Keep it up, demons love it.",0.6726,0.5248,0.5694,0.0949,0.1433
229,CNN,History in the making.,0.6796,0.5078,0.5930,0.0799,0.1565
43,CNN,25th amendment now!,0.3652,0.2041,0.3290,0.0637,0.1413
401,Fox News,they never show the truth,0.4267,0.2316,0.3672,0.0602,0.1581
486,Fox News,She couldnt come up with one.,0.3190,0.1488,0.2078,0.0566,0.1129
477,Fox News,"Idiot, hand puppet",0.5684,0.3142,0.4255,0.0470,0.1881
70,CNN,So Proud of Trump!,0.4788,0.2179,0.4516,0.0445,0.1780
180,CNN,Joy love and peace to all,0.4720,0.2308,0.4160,0.0435,0.1797
197,CNN,What a ginger dork,0.3337,0.1642,0.3103,0.0382,0.1530
253,CNN,The next alantis,0.3778,0.1683,0.3169,0.0380,0.1591


In [37]:
# Most toxic comments
most_toxic = scored.nlargest(10, "score_toxicity")[
    ["channel_name", "text_clean", "score_toxicity", "score_hate_racism", "score_harassment"]
]
print("Top 10 most toxic comments:")
most_toxic

Top 10 most toxic comments:


,channel_name,text_clean,score_toxicity,score_hate_racism,score_harassment
271,CNN,So sorry 🙏,0.9702,0.6913,0.8789
451,Fox News,Love her response.,0.9660,0.7732,0.8881
372,Fox News,Stole the money,0.9654,0.7869,0.8806
483,Fox News,Bush league reporter,0.9549,0.8265,0.8782
408,Fox News,What a loser,0.9523,0.7261,0.8772
419,Fox News,You're a Russian bot.,0.9516,0.6649,0.8196
30,CNN,Why why why,0.9496,0.4204,0.7748
5,CNN,No more war 😂,0.9440,0.8495,0.8620
111,CNN,Long live Persia 🎉,0.9173,0.5517,0.7255
355,Fox News,Seriously doubt it.,0.9112,0.6941,0.8119


## 7. Save Scored Dataset

In [38]:
# Drop mc_samples for the main parquet (save space), keep in separate file
scored_slim = scored.drop(columns=["mc_samples"])
out_path = DATA_DIR / "comments_scored.parquet"
scored_slim.to_parquet(out_path, index=False)

# Save full version with MC samples for uncertainty analysis
out_full = DATA_DIR / "comments_scored_full.parquet"
scored.to_parquet(out_full, index=False)

print(f"Saved:")
print(f"  {out_path}  ({out_path.stat().st_size/1024:.0f} KB)  — slim, for dashboard")
print(f"  {out_full}  ({out_full.stat().st_size/1024:.0f} KB)  — full MC samples")
print()
print("Column summary of comments_scored.parquet:")
print(scored_slim.dtypes.to_string())

Saved:
  C:\Users\owner\Downloads\data_scientist_porfolio\YoutubeCommentSection\data\processed\comments_scored.parquet  (113 KB)  — slim, for dashboard
  C:\Users\owner\Downloads\data_scientist_porfolio\YoutubeCommentSection\data\processed\comments_scored_full.parquet  (185 KB)  — full MC samples

Column summary of comments_scored.parquet:
content_id                            object
platform                              object
video_id                              object
parent_id                             object
text_raw                              object
text_clean                            object
word_count                             int64
lang                                  object
like_count                             int64
reply_count                            int64
published_at             datetime64[ns, UTC]
collected_at             datetime64[ns, UTC]
channel_id                            object
channel_name                          object
channel_category            

## Next Steps

1. **Re-run this notebook** after full GPU training (notebook 03 with `QUICK_TEST=False`) for better scores
2. **M3 — Streamlit dashboard** reads `comments_scored.parquet` → shows toxicity explorer, channel breakdown, uncertainty view
3. **M4 — Gold labeling** — use high-uncertainty comments above as labeling queue in the Streamlit UI
4. **M5 — Deploy** — push ingestion + scoring to Cloud Run, BigQuery replaces the local parquet